<a href="https://colab.research.google.com/github/jazkre/01RAD/blob/main/R/01RAD_Ex10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 01RAD Exercise 10

* Outliers
* Leverages
* Influence points and Influence Measures


zaujíma nás ako nájsť významné body (outliery, pákové b. ...) : ako ich nájsť, čo znamenajú, ako ovplyvňuju našu regresiu, ako s nimi pracovať, čo s nimi?


In [ ]:
# get requirements for this exercise (and previous ones)
list_of_packages <- c("tidyverse", "lattice", "MASS","graphics","psych",
                      "nortest","lmtest","leaps","ISLR","robustbase","ggpubr","gridExtra", "car","GGally") # "car","GGally"
missing_packages <- list_of_packages[!(list_of_packages %in% installed.packages()[,"Package"])]
if(length(missing_packages)) install.packages(missing_packages)
lapply(list_of_packages, library, character.only = TRUE)


In [ ]:

######################
# check our settings
getwd()
#print(.libPaths())
#print(sessionInfo())
#print(version)

# delete graphics
#dev.off()
#par(mfrow = c(1, 1))
set.seed(4242)


Changing data points and play with lin regression


In [ ]:

#Simulate data Y = beta1 + X2*beta2  + X3*beta3  + X4*beta4 + e
# e ~ N(0,4)
n=100
p=4
X0    = rep(1, times=n)
X1    = rnorm(n,20,3)
X2    = 10+rexp(n,0.1)
X3    = 5+rbinom(n,15,0.2)
beta0 = matrix(c(5,3,2,-5),nrow =4 , ncol = 1)
e     = rnorm(n,0,4)
X0    = cbind(X0,X1,X2,X3)
Y     = X0%*%beta0 + e
X     = cbind(X1,X2,X3)


β nepoznáme


In [ ]:

# Create data.frame
data0 = data.frame(cbind(Y,X))
names(data0) = c("Y","X1","X2","X3")
summary(data0)
beta0


In [ ]:

pairs(data0)
pairs.panels(data0,smooth=F,lm=T,scale=T,pch=20)


In [ ]:

# LM model 0
m0 = lm(Y ~ X1 + X2 + X3, data = data0)
summary(m0)
op <- par(mfrow=c(2,3))
plot(m0,which=1)
plot(m0,which=2)
plot(m0,which=3)
plot(m0,which=4)
plot(m0,which=5)
plot(m0,which=6)
par(op)


In [ ]:

par(mfrow=c(1,1))
spreadLevelPlot(m0, col="black")
avPlots(m0)


In [ ]:

# we know real error terms - we can sort them and compare with  residual plots
sort(abs(e),index.return = T,decreasing = T)
#
sort(abs(resid(m0)),index.return = T,decreasing = T)


zoradíme dáta podľa disturbancií (e)

pak podľa residuí

In [ ]:

## Simple Regression ##
# Add good outlying point
X_0 = rbind(X,c(max(X[,1])+20,max(X[,2])+30,max(X[,3])+20))
Y_0 = cbind(rep(1, times=(n+1)),X_0)%*%beta0+c(e,rnorm(1,0,4))
par(mfrow = c(1, 1))
plot(Y_0~X_0[,2],xlab = "X",ylab = "Y", main = "Simple Regression with at least one influential point")
dim(X_0)


In [ ]:

# Simple Regression - only X2 as independent variable
m_0 = lm(Y_0 ~ X_0[,2])
abline(m_0)
pairs.panels(cbind(Y_0,X_0),smooth=F,lm=T,scale=T,pch=20)



In [ ]:

summary(m_0)
op <- par(mfrow=c(2,2))
plot(m_0)
par(op)  


ten pridaný bod (outlier) nedokážeme odhaliť len pomocou residui - tam sa neprejavi



In [ ]:

par(mfrow = c(1, 1))
spreadLevelPlot(m_0, col="black")
avPlots(m_0)


In [ ]:

# in multivariable regression
avPlots(lm(Y_0 ~ X_0))
summary(lm(Y_0 ~ X_0))

ten dolný obrázok scatterplotu : tam to vidíme. to vplyvne pozorovanie

In [ ]:
# Let's play with this added point

# Change its Y value
X_0 = rbind(X,c(max(X[,1])+1,max(X[,2])+1,max(X[,3])+1))
Y_0 = rbind(Y,1)
par(mfrow = c(1, 1))
plot(Y_0~X_0[,2],xlab = "X",ylab = "Y", main = "Simple Regression with at least one influential point")
dim(X_0)
m_1Y = lm(Y_0 ~ X_0[,2])
abline(m_1Y)
pairs.panels(cbind(Y_0,X_0),smooth=F,lm=T,scale=T,pch=20)




In [ ]:

summary(m_1Y)
op <- par(mfrow=c(2,2))
plot(m_1Y)
par(op)  



posunuli sme odozvu a teraz sa ten "špatný bod premietol aj do rezidui. týmto by sme ho mali odhaliť

čo ale teraz s ním?

musíme zistiť, či tie dáta patria do datasetu. môžu to byť len zlé jednotky, alebo iná skupina (dá sa pridať premenná označujúca skupinu)

In [ ]:

spreadLevelPlot(m_1Y, col="black")
avPlots(m_1Y)


In [ ]:

# in multivariable regression
avPlots(lm(Y_0 ~ X_0))
summary(lm(Y_0 ~ X_0))




In [ ]:

# Change its X value
X_0 = rbind(X,c(1,1,1),c(1,1,1),c(1,1,1),c(1,1,1),c(1,1,1),c(1,1,1),c(1,1,1),c(1,1,1))
Y_0 = rbind(Y,125,126,124,127,128,126,123,130)
par(mfrow = c(1, 1))
plot(Y_0~X_0[,2],xlab = "X",ylab = "Y", main = "Simple Regression with at least one influential point")
dim(X_0)
m_1X = lm(Y_0 ~ X_0[,2])
abline(m_1X)

summary(m_1X)
op <- par(mfrow=c(2,2))
plot(m_1X)
par(op)  

par(mfrow = c(1, 1))
spreadLevelPlot(m_1X, col="black")
avPlots(m_1X)




In [ ]:
# Try to add more outliers next to each other

# compare with  multivariable regression
avPlots(lm(Y_0 ~ X_0))
summary(lm(Y_0 ~ X_0))




In [ ]:

# Add more leverages in more regressors         
X_1 = X
X_1[c(11,21,31),]
X_1[11,1] = 2*X_1[11,1]
X_1[21,2] = 2*X_1[21,2]
X_1[31,3] = 2*X_1[31,3]


In [ ]:

op <- par(mfrow=c(1,3))
plot(Y~X_1[,1],xlab = "X1",ylab = "Y", main = "Scatterplot Y~X1")
plot(Y~X_1[,2],xlab = "X2",ylab = "Y", main = "Scatterplot Y~X2")
plot(Y~X_1[,3],xlab = "X3",ylab = "Y", main = "Scatterplot Y~X3")
par(op)


In [ ]:

data_1 <- data.frame(cbind(Y,X_1))
names(data_1) <- c("Y","X1","X2","X3")
pairs.panels(data_1,smooth=F,lm=T,scale=T,pch=20)



zo scatterplottov to nie je uplne dobre vidieť. 

outliery nemusia byť hneď vidieť ´. Ale model to odhali

In [ ]:

m1 = lm(Y ~ X_1)
summary(m1)
op <- par(mfrow=c(2,3))
plot(m1,which=1)
plot(m1,which=2)
plot(m1,which=3)
plot(m1,which=4)
plot(m1,which=5)
plot(m1,which=6)
par(op)



In [ ]:

op <- par(mfrow=c(1,1))
spreadLevelPlot(m1, col="black")
avPlots(m1)


In [ ]:

# Add more outliers in response 

Y_2 = Y
Y_2[c(12,22,32)]
Y_2[12] =  2*Y_2[12]
Y_2[22] =  2*Y_2[22]
Y_2[32] =  2*Y_2[32]


In [ ]:

m2 = lm(Y_2 ~ X)
summary(m2)
op <- par(mfrow=c(2,3))
plot(m2,which=1)
plot(m2,which=2)
plot(m2,which=3)
plot(m2,which=4)
plot(m2,which=5)
plot(m2,which=6)
par(op)



v reziduach (1.obr) to vidím -> vidím, že tie dáta sú niečím iné

zhoršil sa aj performance modelu (resid.error)

In [ ]:

op <- par(mfrow=c(1,1))
spreadLevelPlot(m2, col="black")
avPlots(m2)

In [ ]:
# Model with 3 leverages and 3 outliers

m3 = lm(Y_2 ~ X_1,x=T,y=T,model=T)


In [ ]:

summary(m3)
op <- par(mfrow=c(2,2))
plot(m3)
par(op)


In [ ]:

op <- par(mfrow=c(2,3))
plot(m3,which=1)
plot(m3,which=2)
plot(m3,which=3)
plot(m3,which=4)
plot(m3,which=5)
plot(m3,which=6)
par(op)



In [ ]:

op <- par(mfrow=c(1,1))
spreadLevelPlot(m3, col="black")
avPlots(m3)
summary(lm(Y_2 ~ X_1))



### influence.measures()

Produces a class "infl" object tabular display showing  the DFBETAS for each model variable, DFFITS, covariance ratios, Cook's distances and the diagonal elements of the hat matrix.

In [ ]:
summary(influence.measures(m1))
summary(influence.measures(m2))
summary(influence.measures(m3))

dfb.1 dfbetas : 

In [ ]:

influence.measures(m3)

In [ ]:

# diagonal elements of hat matrix
X_1tX_1  <- t(X_1) %*% X_1 
H <- X_1 %*% solve(X_1tX_1) %*% t(X_1)    
H[1:5,1:5]
tr(H)


In [ ]:

#Residual projection matrix
M <- diag(dim(X_1)[1]) - H
# diagonal elements of hat matrix without intercept
diag(H)
hat(X_1,intercept = F)
# diagonal elements of hat matrix with intercept
hat(X_1,intercept = T) 
hatvalues(m3)



In [ ]:

# leverage point
# A diagonal element hat matrix H (h_{i,i}) is called the leverage of the i-th observation.
# i-th observation is a leverage point if h_i,i >  3r\n = sum(diag(H))\n
r = sum(hatvalues(m3)) # = rank(X)
n = length(Y)
leverage_level = 3*r/n #pravidlo palca
leverage_level
sum(hatvalues(m3) > leverage_level)
data_1[hatvalues(m3) > leverage_level,]
sort(hatvalues(m3))
data_1[hatvalues(m3) > 0.1,]

#dostaneme to samé ako summary(influence.measures(m3))


In [ ]:

#standardized residuals (Normovana rezidua)
rstandard(m3)
#abs(rstandard(m3)-resid(m3))


In [ ]:

sigma_hat    = summary(m3)$sigma
res          = summary(m3)$residuals
h            = hatvalues(m3)
m            = 1-hatvalues(m3)
res_standard = res/(sigma_hat*sqrt(m))

sort(rstandard(m3))[1:10]
sort(res_standard)[1:10]



In [ ]:

# Studentized residuals (studentizovana rezidua)
rstudent(m3)
sort(rstudent(m3))


In [ ]:

res_deleted = resid(m3)/m  # Deleted residual Yi - hatY[i]
#           = residuals(m3) / (1 - hatvalues(m3))
SSe         = sum(resid(m3)^2)
SSe_deleted = SSe - (resid(m3)^2)/m
SSe_deleted == SSe - (SSe/(n-r))*res_standard^2
MSe_deleted = SSe_deleted/(n-1-r)


In [ ]:
res_student  = resid(m3)/(sqrt(MSe_deleted*m))
res_student2 = sqrt((n-r-1)/(n-r-res_standard^2))*res_standard

In [ ]:
sort(rstudent(m3))[1:10]

In [ ]:
sort(res_student)[1:10]

In [ ]:
sort(res_student2)[1:10]

In [ ]:

sort(abs(rstudent(m3)),decreasing = T)[1:10]




porovnáme, možeme urobiť ttest

In [ ]:

#dfbeta displays effect on coefficients of deleting each observation in turn
dfbeta(m3)
dfbetaPlots(m3)

apply(abs(dfbeta(m3)), 2, summary)




In [ ]:

#dfbetas displayseffect on coefficients of deleting each observation in turn,
#  standardized by a deleted estimate of the coefficient standard error
dfbetas(m3)
dfbetasPlots(m3)

apply(abs(dfbetas(m3)), 2, summary)

dfbetas(m3)[abs(dfbetas(m3)[, 2])>0.1,]
dfbetas(m3)[abs(dfbetas(m3)[, 3])>0.1,]
dfbetas(m3)[abs(dfbetas(m3)[, 4])>0.1,]
# i-th observation is influential with respect to the OLS of the j-th regression coefficient
# if abs(dfbetas_ij) > 1


In [ ]:

# handy 
#dfbetas_hand = (t((resid(m3)/m)*(solve(t(m3$x)%*%m3$x))%*%t(m3$x))/sqrt(MSe_deleted))/sqrt(diag(solve(t(m3$x)%*%m3$x)))



In [ ]:

# dffits statistics
dffits(m3)
sort(abs(dffits(m3)),decreasing = T)[1:10]


In [ ]:

# handy
dffits_hand = sqrt(hatvalues(m3)/(1-hatvalues(m3)))*rstudent(m3) # = (hatY - hatY[i])/sqrt(MSe[-i]*h_tt)
sort(abs(dffits_hand),decreasing = T)[1:10]


In [ ]:

#observation excessively influences  if abs(dffits) > 3*sqrt(r/(n-r))
dffits_level = 3*sqrt(r/(n-r))
data_1[abs(dffits(m3)) > dffits_level,]

# S in DFBETAS and DFFITTS means SCALED!!



In [ ]:

# covratio statistics: 
# ratio of determinants of covariance matrices of full and deleted (one-leave-out) model
covratio(m3)
sort(covratio(m3))
#covratio_hand = (1/m)*  sqrt((n-r-1)/(n-r-res_standard^2))*res_standard
# Infulence.measures shows the i-th observation as "influencing" if 
# abs(1-covratio) > 3*r/(n-r) 
# summary(influence.measures(m3))



In [ ]:

#The i-th studentized residual follows the Student t-distribution with (n-r-1) DF
# cooks distance
cooks.distance(m3)
sort(cooks.distance(m3),decreasing = T)


In [ ]:

cooks_hand = (h/(r*m))*rstandard(m3)^2
# h_ii/m_ii is high for leverages points 
# rstandard(m3)^2 is high for outliers


In [ ]:

sort(cooks.distance(m3),decreasing = T)[1:10]
sort(cooks_hand,decreasing = T)[1:10]
# Infulence.measures shows the i-th observation as "influencing" if 
# Cook_dist > qf(0.5,r,n-r)


In [ ]:

plot(m3,which=5)
plot(m3,which=6)

### Summary 

* Don't panic if something is influential ;)
* Occurence of outliers or leverage points does not necessarily mean a problem
* Never delete influence points (outliers, leverages) without reason 
* To have "nicer" residuals  is not a reason
* Outliers have to be explored and new model "defended and EXPLAINED"


## Exercise

**Use Hertzsprung-Russell Diagram Data of Star Cluster CYG OB1**


In [ ]:
? starsCYG
summary(starsCYG)
attach(starsCYG)
pairs.panels(starsCYG,smooth=F,lm=T,scale=T,pch=20)


In [ ]:

# HW: What does it mean if we swap independent and dependent varialbe?
m_Te   <- (lm(log.Te    ~ log.light,data=starsCYG))
m_light <-(lm(log.light ~ log.Te,   data=starsCYG))
summary(m_Te)
summary(m_light)


In [ ]:

summary(influence.measures(m_Te))
summary(influence.measures(m_light))


In [ ]:

round(sort(hatvalues(m_Te),decreasing = T)[1:10],4)
round(sort(hatvalues(m_light),decreasing = T)[1:10],4)
r = sum(hatvalues(m_Te))
n = 47
r/n



In [ ]:

round(sort(dffits(m_Te),decreasing = T)[1:10],4)
round(sort(dffits(m_light),decreasing = T)[1:10],4)
3*sqrt(r/(n-r))
starsCYG[abs(dffits(m_Te)) > 3*sqrt(r/(n-r)),]
starsCYG[abs(dffits(m_light)) > 3*sqrt(r/(n-r)),]


In [ ]:

op <- par(mfrow=c(3,4))
plot(m_Te,which=1)
plot(m_Te,which=2)
plot(m_Te,which=3)
plot(m_Te,which=4)
plot(m_Te,which=5)
plot(m_Te,which=6)
plot(m_light,which=1)
plot(m_light,which=2)
plot(m_light,which=3)
plot(m_light,which=4)
plot(m_light,which=5)
plot(m_light,which=6)
par(op)

# try to find outliers, and use all identifiers as leverage, cook, dfbeta, dffit, rstudent, ...


In [ ]:

# 2) Use Credit_out data - compare with model with credit date used two lectres ago

Credit <- read.table("https://raw.githubusercontent.com/francji1/01RAD/main/data/Credit.csv",header=TRUE,sep=",")
head(Credit)

In [ ]:

Credit_out <- read.table("https://raw.githubusercontent.com/francji1/01RAD/main/data/Credit_out.csv",header=TRUE,sep=",")
head(Credit_out)

#Credit     <- read.table("data/Credit.csv", sep =",", header = T )
#Credit_out <- read.table("data/Credit_out.csv", sep =",", header = T )


In [ ]:

dim(Credit)
dim(Credit_out)
Credit_out[398:408,]
# Analyze model with dataset with and without zeros in response varaible Balance
m_c = lm(Balance~(.),data=Credit)
m_co =lm(Balance~(.),data=Credit_out)
summary(m_c)
summary(m_co)


In [ ]:

m_c = lm(Balance~Income+Limit+Cards+Student,data=Credit[Credit$Balance != 0,])
m_co =lm(Balance~Income+Limit+Cards+Student,data=Credit_out[Credit$Balance != 0,])


In [ ]:
summary(m_c)

In [ ]:
summary(m_co)

In [ ]:
summary(influence.measures(m_c))

In [ ]:
summary(influence.measures(m_co))

In [ ]:

op <- par(mfrow=c(3,4))
plot(m_c,which=1)
plot(m_c,which=2)
plot(m_c,which=3)
plot(m_c,which=4)
plot(m_c,which=5)
plot(m_c,which=6)
plot(m_co,which=1)
plot(m_co,which=2)
plot(m_co,which=3)
plot(m_co,which=4)
plot(m_co,which=5)
plot(m_co,which=6)
par(op)

In [ ]:

A <-ggplot(m_c, aes(.fitted, .resid)) +
  geom_point() +
  geom_smooth(se = T)

B <-ggplot(m_c, aes(.fitted, sqrt(abs(.stdresid)))) +
  geom_point() +
  geom_smooth(se = T)

C <- ggplot(m_c) +
  stat_qq(aes(sample = .stdresid)) +
  geom_abline()

D <- ggplot(m_c, aes(seq_along(.cooksd), .cooksd)) +
  geom_col()

E <- ggplot(m_c, aes(.hat, .stdresid)) +
  geom_vline(size = 2, colour = "white", xintercept = 0) +
  geom_hline(size = 2, colour = "white", yintercept = 0) +
  geom_point(aes(size = .cooksd)) +
  geom_smooth(se = FALSE, size = 0.5)

F <- ggplot(m_c, aes(.hat, .cooksd)) +
  geom_vline(xintercept = 0, colour = NA) +
  geom_abline(slope = seq(0, 3, by = 0.5), colour = "white") +
  geom_smooth(se = FALSE) +
  geom_point(aes(size = .cooksd / .hat)) +
  scale_size_area()


In [ ]:

#library(gridExtra)
# How to use grid.arrange
# https://cran.r-project.org/web/packages/egg/vignettes/Ecosystem.html
grid.arrange(A, B, C, D, E, F, nrow = 2)


In [ ]:

#library(ggpubr)
# How to use ggpubr
# http://www.sthda.com/english/articles/24-ggpubr-publication-ready-plots/81-ggplot2-easy-way-to-mix-multiple-graphs-on-the-same-page/
ggarrange(A, B, C,D,E, F + rremove("x.text"), 
          labels = c("A", "B", "C", "D", "E", "F"),
          ncol = 3, nrow = 2)
